In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Layer, Reshape, InputLayer
from alibi_detect.od import OutlierVAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from glob import glob
import os

ImportError: Traceback (most recent call last):
  File "C:\Users\pc\anaconda3\envs\notebook\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python310.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
import matplotlib.pyplot as plt

img_list = glob('C:/Users/pc/Desktop/road/512/*.jpg')
train_img_list, val_img_list = train_test_split(img_list, test_size=0.1, random_state=2023)

def img_to_np(fpaths, resize=True):  
    img_array = []
    for fname in fpaths:
      try:
        img = Image.open(fname).convert('RGB')
        if(resize): img = img.resize((512, 512))
        img_array.append(np.asarray(img))
      except:
        continue
    images = np.array(img_array)
    return images

x_train = img_to_np(train_img_list)
x_train = x_train.astype(np.float32) / 255.

x_val = img_to_np(val_img_list)
x_val = x_val.astype(np.float32) / 255.

print(x_train.shape)
print(x_val.shape)

In [ ]:
# check image
plt.imshow(x_train[0])

In [ ]:
latent_dim = 2048

encoder_net = tf.keras.Sequential([
    InputLayer(input_shape=(512, 512, 3)),
    Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
    Conv2D(1024, 4, strides=2, padding='same', activation=tf.nn.relu)
])

decoder_net = tf.keras.Sequential([
    InputLayer(input_shape=(latent_dim,)),
    Dense(128 * 128 * 512),
    Reshape(target_shape=(128, 128, 512)),
    Conv2DTranspose(128, 128, strides=2, padding='same', activation=tf.nn.relu),
    Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
])

od = OutlierVAE(
    threshold=.0004,
    score_type='mse',
    encoder_net=encoder_net,
    decoder_net=decoder_net,
    latent_dim=latent_dim,
)

In [ ]:
od.fit(
    x_train,
    epochs=100,
    verbose=True
)

In [ ]:
from alibi_detect.saving import save_detector

save_file_path = 'file path for saving weights'
save_detector(od, save_file_path)

In [ ]:
from alibi_detect.saving import load_detector

od2 = load_detector(save_file_path)

In [ ]:
idx = 12
x = x_train[idx].reshape(1, 128, 128, 3)
x_recon = od.vae(x).numpy()

fig, axes = plt.subplots(1, 2, figsize=(16, 10))

axes[0].imshow(x.squeeze())
axes[1].imshow(x_recon.squeeze())

In [ ]:
od_preds = od.predict(
    x_val,
    outlier_type='instance',
    return_feature_score=True,
    return_instance_score=True
)

target = np.zeros(x_val.shape[0],).astype(int)
labels = ['normal', 'outlier']
plot_instance_score(od_preds, target, labels, od.threshold)

In [ ]:
x_recon = od.vae(x_val).numpy()

plot_feature_outlier_image(
    od_preds,
    x_val,
    X_recon=x_recon,
    max_instances=5,
    outliers_only=False
)

In [ ]:
plot_feature_outlier_image(
    od_preds,
    x_val,
    X_recon=x_recon,
    max_instances=5,
    outliers_only=True
)

In [ ]:
test_img_list = glob('test image file path')

x_test = img_to_np(test_img_list[:10])
x_test = x_test.astype(np.float32) / 255.

print(x_test.shape)

In [ ]:
od_preds = od.predict(
    x_test,
    outlier_type='instance',
    return_feature_score=True,
    return_instance_score=True
)

target = np.zeros(x_test.shape[0],).astype(int)
labels = ['normal', 'outlier']
plot_instance_score(od_preds, target, labels, od.threshold)

In [ ]:
plt.imshow(x_test[9])

In [ ]:
x_recon = od.vae(x_test).numpy()

plot_feature_outlier_image(
    od_preds,
    x_test,
    X_recon=x_recon,
    max_instances=10,
    outliers_only=False
)

In [ ]:
x_recon = od.vae(x_test).numpy()

plot_feature_outlier_image(
    od_preds,
    x_test,
    X_recon=x_recon,
    max_instances=5,
    outliers_only=True
)